# Preparação da tabela de OS

### Importando as bibliotecas necessárias

In [ ]:
import numpy as np
import pandas as pd

### Lendo o dataframe de OS

In [ ]:
path = "../../../dados/"
pathIn = path + "input/"

pathUtil = path + "util/"

df_original = pd.read_excel(
    pathIn + "IW49_OP_Executadas_2020-a-28jun24.xlsx", sheet_name="Sheet1"
)

df_coord = pd.read_csv(pathUtil + "instalacao_coordenada.csv")

df_copy_ori = df_original.copy()
df_copy_coord = df_coord.copy()

In [ ]:
df_original = df_copy_ori
df_coord = df_copy_coord

df_original.info()

### Tirando OS com HH igual a 0

In [ ]:
df_original = df_original.loc[
    (
        (df_original["Trabalho previsto"] != 0)
        | (df_original["Trabalho"] != 0)
        | (df_original["Trabalho real"] != 0)
    )
]

### Removendo OS sem Local de Instalação

In [ ]:
df_original = df_original.loc[df_original["Local de instalação"] != ""]

### Removendo OS sem data de início

In [ ]:
df_original = df_original.loc[
    (df_original["1ª data de início"].notna())
    | (df_original["Última data início"].notna())
    | (df_original["Data do início real"].notna())
]

### Reduzindo a quantidade de siglas do Local de Instalação

In [ ]:
def local(linha):
    string = str(linha)
    parts = string.split("-")

    if len(parts) > 3:
        result = "-".join(parts[:3])
    else:
        result = string

    return result


df_original["Local de instalação"] = df_original["Local de instalação"].apply(local)

### Removendo OS com Local de instalação sem coordenada

In [ ]:
print(df_original["Local de instalação"].nunique())
print(df_coord["Local de instalação"].nunique())

linhas_nao_presentes = df_coord[
    ~df_coord["Local de instalação"].isin(df_original["Local de instalação"])
]
print("Linhas nao presentes: ")
print(linhas_nao_presentes["Local de instalação"].unique())

df_original = df_original[
    df_original["Local de instalação"].isin(df_coord["Local de instalação"])
]

print(df_original["Local de instalação"].nunique())

### Criando Data de início e fim

In [ ]:
def data_real(linha):
    # esta funcao permite que haja uma data fim maior do que a data inicio em casos especificos onde o ultimo fim ainda náo foi escrito.
    primeiro_inicio = linha["1ª data de início"]
    ultimo_inicio = linha["Última data início"]
    inicio_real = linha["Data do início real"]

    primeiro_fim = linha["1ª data fim"]
    ultimo_fim = linha["Última data final"]

    if (inicio_real is pd.NaT) and (ultimo_inicio is pd.NaT):
        linha["Data inicio"] = primeiro_inicio

    elif (inicio_real is pd.NaT) and (ultimo_inicio is not pd.NaT):
        linha["Data inicio"] = ultimo_inicio

    else:
        linha["Data inicio"] = inicio_real

    if ultimo_fim is pd.NaT:
        linha["Data fim"] = primeiro_fim

    else:
        linha["Data fim"] = ultimo_fim

    return linha


df_original = df_original.apply(data_real, axis=1)

### Criando a coluna do HH final do agrupamento

In [ ]:
def hh_final(linha):
    if linha["Trabalho real"] == 0:
        return linha["Trabalho previsto"]
    else:
        return linha["Trabalho real"]


df_original["HH final"] = df_original.apply(hh_final, axis=1)

### Trocando os HH em minutos para horas

In [ ]:
def convert_to_hr(linha):
    if linha["Unidade do trabalho"] == "MIN":
        linha["HH final"] /= 60.0
        linha["Unidade do trabalho"] = "HR"

    return linha


df_original = df_original.apply(convert_to_hr, axis=1)

### Montando a tabela com as colunas necessárias

In [ ]:
df_cortado = df_original[
    [
        "Ordem",
        "Operação",
        "Confirmação",
        "Local de instalação",
        "Data inicio",
        "Data fim",
        "HH final",
    ]
]
df = df_cortado.sort_values(by=["Data inicio", "Local de instalação"])

print(df.head(5))

### Exportando o DataFrame

In [ ]:
df.to_csv(pathUtil + "IW49_Executadas_processado.csv", index=False)